# Question 3: Has COVID had an impact during holiday seasons? 

### COVID Start & End Date:
----------------------------------------------------------------
* Start Date: January 24, 2020
* End Date: February 24, 2022 => [Article from Chicago.gov](https://www.chicago.gov/city/en/sites/covid-19/home/health-orders.html)

In [1]:
import pandas as pd
import os
from zipfile import ZipFile
import holidays
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_style('dark')
plt.rcParams['font.size'] = 10
plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.figsize'] = (10, 4)

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [3]:
# Zip File Extraction
    # ? Insead of extracting one zip file at a given time, we will extract all the zip files using a for loop
def unzip_files(zip_path, extract_path):
    for zip_path in zip_paths:
        with ZipFile (zip_path, 'r') as zObject:
        # * Extracting all memebters of the zip files into a specific location
            zObject.extractall(
                path = extract_path
            )
# * Usage zip function - Variables:
zip_paths = ["/Users/galbeeir/Desktop/git/Gal Crime Analysis/crime_analysis/filtered_1.zip", "/Users/galbeeir/Desktop/git/Gal Crime Analysis/crime_analysis/filtered_2.zip"]
extract_path = "/Users/galbeeir/Desktop/git/Gal Crime Analysis/crime_analysis/"

# * Executing the zip function
unzip_files(zip_paths, extract_path)


In [4]:
# * List of directories
datasets_1 = "/Users/galbeeir/Desktop/git/Gal Crime Analysis/crime_analysis/CSVs"

# * List all the files from the directory
file_list_1 = os.listdir(datasets_1)

# * Merge all files stored in file_list
crime_df = pd.DataFrame()

# * Merge all files stored in file_list_1
for file in file_list_1:

    if file.endswith(".csv"):
        file_path = os.path.join(datasets_1, file)
        df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')
        crime_df = pd.concat([crime_df, df])

     
crime_df.head()


/var/folders/06/47q5ms9540g4mnz8nnnyl32m0000gn/T/ipykernel_47261/1307425474.py:15: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')
/var/folders/06/47q5ms9540g4mnz8nnnyl32m0000gn/T/ipykernel_47261/1307425474.py:15: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')
/var/folders/06/47q5ms9540g4mnz8nnnyl32m0000gn/T/ipykernel_47261/1307425474.py:15: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')
/var/folders/06/47q5ms9540g4mnz8nnnyl32m0000gn/T/ipykernel_47261/1307425474.py:15: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')
/var/folders

,Unnamed: 0,ID,Case Number,Date,Year,Updated On,Block,Primary Type,Location Description,Arrest,Domestic,Police District,Ward,Community Area,Latitude,Longitude
0,807159,12581817,JF100052,2022-01-01 00:00:00,2022,2023-01-03 15:46:28,080XX S CRANDON AVE,BATTERY,RESIDENCE,True,True,4,8.0,South Chicago,41.749260,-87.56863
1,807160,12581845,JF100074,2022-01-01 00:00:00,2022,2023-01-03 15:46:28,013XX S MICHIGAN AVE,BATTERY,APARTMENT,True,True,1,3.0,Near South Side,41.865147,-87.62410
2,807161,12581875,JF100038,2022-01-01 00:00:00,2022,2023-01-03 15:46:28,116XX S CHURCH ST,CRIMINAL DAMAGE,RESIDENCE,False,False,22,34.0,Morgan Park,41.681515,-87.67086
3,807162,12581981,JF100073,2022-01-01 00:00:00,2022,2023-01-03 15:46:28,133XX S CORLISS AVE,CRIMINAL DAMAGE,APARTMENT,False,False,5,9.0,Riverdale,41.653637,-87.60207
4,807163,12582064,JF100392,2022-01-01 00:00:00,2022,2023-01-03 15:46:28,019XX S HALSTED ST,THEFT,VEHICLE NON-COMMERCIAL,False,False,12,11.0,Lower West Side,41.856380,-87.64660


In [5]:
crime_df['Date'] = pd.to_datetime(
    crime_df['Date'],
    errors = 'coerce',
    infer_datetime_format=True, 
)

In [6]:
crime_df['Updated On'] = pd.to_datetime(
    crime_df['Updated On'],
    errors = 'coerce',
    infer_datetime_format=True, 
    format ='%T'
)

In [7]:
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028014 entries, 0 to 202875
Data columns (total 16 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   Unnamed: 0            1028014 non-null  int64         
 1   ID                    1028014 non-null  int64         
 2   Case Number           1028014 non-null  object        
 3   Date                  1028014 non-null  datetime64[ns]
 4   Year                  1028014 non-null  int64         
 5   Updated On            1028014 non-null  datetime64[ns]
 6   Block                 1028014 non-null  object        
 7   Primary Type          1028014 non-null  object        
 8   Location Description  1028014 non-null  object        
 9   Arrest                1028014 non-null  bool          
 10  Domestic              1028014 non-null  bool          
 11  Police District       1028014 non-null  int64         
 12  Ward                  1028014 non-null  obj

In [8]:
# Get the list of holidays for a specific country/region
us_holidays = holidays.US()

In [9]:
crime_df['Is_Holiday'] = crime_df['Date'].apply(lambda x: x in us_holidays)

In [10]:
crime_df.drop('Unnamed: 0', axis=1, inplace=True)

In [11]:
# Define a function to receive the holiday's name
def get_holiday_name(date):
    for holiday_date, holiday_name in us_holidays.items():
        if date.date() == holiday_date:
            return holiday_name
    return None

In [12]:
crime_df['Holiday_name'] = crime_df['Date'].apply(get_holiday_name)

In [1]:
crime_df.sort_values('Year', inplace=True)

NameError: name 'crime_df' is not defined

In [14]:
crime_df.head()

,ID,Case Number,Date,Year,Updated On,Block,Primary Type,Location Description,Arrest,Domestic,Police District,Ward,Community Area,Latitude,Longitude,Is_Holiday,Holiday_name
156498,11786468,JC383237,2019-08-07 21:17:00,2019,2019-08-14 16:17:18,087XX S STONY ISLAND AVE,NARCOTICS,STREET,True,False,4,8.0,Calumet Heights,41.736465,-87.58521,False,None
104326,11709621,JC289778,2019-06-03 05:00:00,2019,2019-07-22 16:08:58,015XX W FULLERTON AVE,BURGLARY,SMALL RETAIL STORE,True,False,19,32.0,Lincoln Park,41.925175,-87.66651,False,None
104327,11710759,JC291723,2019-06-03 05:00:00,2019,2019-06-30 15:56:27,049XX W PARKER AVE,OTHER OFFENSE,APARTMENT,False,False,25,31.0,Belmont Cragin,41.930584,-87.74996,False,None
104328,11711168,JC291809,2019-06-03 05:15:00,2019,2019-06-30 15:56:27,032XX N Broadway,DECEPTIVE PRACTICE,DRUG STORE,False,False,19,44.0,Lake View,41.940320,-87.64441,False,None
104329,11711192,JC291807,2019-06-03 05:15:00,2019,2019-06-30 15:56:27,032XX N ASHLAND AVE,THEFT,ATHLETIC CLUB,False,False,19,47.0,Lake View,41.940903,-87.66869,False,None


In [15]:
holiday_df = crime_df.loc[crime_df['Is_Holiday']==True]

In [16]:
holiday_df['Holiday_name'].unique()

array(['Memorial Day', 'Independence Day', "Washington's Birthday",
       "New Year's Day", 'Martin Luther King Jr. Day', 'Veterans Day',
       'Christmas Day', 'Thanksgiving', 'Labor Day', 'Columbus Day',
       'Independence Day (Observed)', 'Christmas Day (Observed)',
       "New Year's Day (Observed)",
       'Juneteenth National Independence Day (Observed)',
       'Juneteenth National Independence Day'], dtype=object)

In [26]:
# Replacing duplicated holidays
    # ? As we can see, the following holidays are duplicates => Independence Day (Observed), Christmas Day (Observed), New Year's Day (Observed), Juneteenth National Independence Day (Observed). 
holiday_df.replace({
    'Independence Day (Observed)': 'Independence Day',
    'Christmas Day (Observed)': 'Christmas Day',
    "New Year's Day (Observed)": "New Year's Day",
    'Juneteenth National Independence Day (Observed)': 'Juneteenth National Independence Day'
}, inplace=True)

/var/folders/06/47q5ms9540g4mnz8nnnyl32m0000gn/T/ipykernel_47261/4089363784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_df.replace({


In [18]:
# See confirmation for the amount of holidays in USA here https://www.commerce.gov/hr/employees/leave/holidays
holiday_df['Holiday_name'].nunique()

11

In [32]:
holiday_df = holiday_df.reset_index(drop=True)

In [35]:
holiday_df.drop('index', axis=1, inplace=True)

In [39]:
holiday_df.head()

,ID,Case Number,Date,Year,Updated On,Block,Primary Type,Location Description,Arrest,Domestic,Police District,Ward,Community Area,Latitude,Longitude,Is_Holiday,Holiday_name
0,11703465,JC282903,2019-05-27 21:00:00,2019,2019-06-30 15:56:27,007XX W 14TH ST,THEFT,STREET,False,False,12,25.0,Near West Side,41.863590,-87.645310,True,Memorial Day
1,11702897,JC282303,2019-05-27 21:00:00,2019,2019-06-30 15:56:27,003XX N PINE AVE,CRIMINAL DAMAGE,STREET,False,False,15,37.0,Austin,41.886642,-87.763405,True,Memorial Day
2,11703024,JC281136,2019-05-27 21:00:00,2019,2019-06-30 15:56:27,045XX N NARRAGANSETT AVE,BATTERY,RESIDENCE,False,True,16,38.0,Portage Park,41.964096,-87.786766,True,Memorial Day
3,11703083,JC282234,2019-05-27 21:00:00,2019,2019-06-30 15:56:27,024XX W Logan Blvd,THEFT,VEHICLE NON-COMMERCIAL,False,False,14,32.0,Logan Square,41.928413,-87.688710,True,Memorial Day
4,11703136,JC281123,2019-05-27 21:00:00,2019,2019-06-30 15:56:27,001XX E HURON ST,BATTERY,HOTEL/MOTEL,False,True,18,42.0,Near North Side,41.894950,-87.623620,True,Memorial Day


### Creating 3 DataFrames (during, post, and prior COVID) based on holiday_df

In [36]:
prior_covid_holiday = holiday_df.loc[holiday_df['Date'] <= "2020-01-24"]

In [37]:
durring_covid_holidays = holiday_df.loc[(holiday_df['Date'] >= "2020-01-24") | (holiday_df['Date'] <= "2022-02-24")]

In [38]:
post_covid_holiday = holiday_df.loc[holiday_df['Date'] >= "2022-02-24"]